In [1]:
import pandas as pd

In [2]:
school_data_to_load='Resources/schools_complete.csv'
student_data_to_load='Resources/clean_students_complete.csv'

school_data_df=pd.read_csv(school_data_to_load)
student_data_df=pd.read_csv(student_data_to_load)

In [3]:
# Merge the school+student data files
school_data_complete_df=pd.merge(student_data_df,school_data_df,on=['school_name','school_name'])
school_data_complete_df.head()

,Unnamed: 0,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,3,Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [4]:
student_count=school_data_complete_df['Student ID'].count()

school_list = school_data_complete_df['school_name'].unique()
school_count=len(school_list)
for school in school_list:
    school_budgets=school_data_complete_df['budget']
district_budget=sum(school_budgets.unique())
average_math_score=school_data_complete_df['math_score'].mean()
average_reading_score=school_data_complete_df['reading_score'].mean()

# Get all the students who are passing math in a new DataFrame.
passing_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70]

# Get all the students that are passing reading in a new DataFrame.
passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]

# Calculate the number of students passing math.
passing_math_count = passing_math["student_name"].count()

# Calculate the number of students passing reading.
passing_reading_count = passing_reading["student_name"].count()

# Calculate the percent that passed math.
passing_math_percentage = passing_math_count / float(student_count) * 100

# Calculate the percent that passed reading.
passing_reading_percentage = passing_reading_count / float(student_count) * 100

# Calculate the students who passed both math and reading.
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)]

passing_math_reading_count=passing_math_reading['student_name'].count()

overall_passing_percentage=passing_math_reading_count/student_count*100

In [5]:
# Create a DataFrame
district_summary_df = pd.DataFrame(
          [{"Total Schools": school_count, 
          "Total Students": student_count, 
          "Total Budget": district_budget,
          "Average Math Score": average_math_score, 
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading_percentage,
        "% Overall Passing": overall_passing_percentage}])



# Format the "Total Students" to have the comma for a thousands separator.
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
# Format the "Total Budget" to have the comma for a thousands separator, a decimal separator and a "$".
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
# Format the columns.
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.0f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.0f}%".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.0f}%".format)

# Display the data frame
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",79.0,81.9,75%,86%,65%


In [6]:
# Determine the school type for the list of schools.
per_school_types = school_data_df.set_index(["school_name"])["type"]

In [7]:
# Calculate the total student count. (Method 1)
per_school_counts= school_data_df.set_index(['school_name'])['size']
per_school_counts

# Calculate the total student count. (Method 2)
per_school_counts = school_data_complete_df["school_name"].value_counts()

In [8]:
# Calculate the total school budget.
per_school_budget = school_data_df.set_index(["school_name"])["budget"]

In [9]:
# Calculate the per capita spending.
per_school_capita = per_school_budget / per_school_counts

In [10]:
# Calculate the math scores.
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()['math_score']

In [11]:
# Calculate the reading scores.
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()['reading_score']

In [12]:
# Set passing scores to 70 or higher
passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

# Create a series of all students passing their math test
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]

# Group the previous series by school_name
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]

# Create a series of all students passing their reading test
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

# Group the previous series by school_name
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100

per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

In [13]:
# Calculate the students who passed both math and reading.
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) 
                                                   & (school_data_complete_df["reading_score"] >= 70)]

per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

In [14]:
# Adding a list of values with keys to create a new DataFrame.

per_school_summary_df = pd.DataFrame({
             "School Type": per_school_types,
             "Total Students": per_school_counts,
             "Total School Budget": per_school_budget,
             "Per Student Budget": per_school_capita,
             "Average Math Score": per_school_math,
           "Average Reading Score": per_school_reading,
           "% Passing Math": per_school_passing_math,
           "% Passing Reading": per_school_passing_reading,
           "% Overall Passing": per_overall_passing_percentage})

per_school_summary_df['Total School Budget']=per_school_summary_df['Total School Budget'].map('${:,.2f}'.format)
per_school_summary_df['Per Student Budget']=per_school_summary_df['Per Student Budget'].map('${:,.2f}'.format)

In [15]:
# Sort and show top five schools.
top_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)

In [16]:
# Sort and show bottom five schools.
bottom_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=True)

In [17]:
# Create a grade level dataframes
ninth_graders = school_data_complete_df[(school_data_complete_df['grade']=='9th')]
tenth_graders = school_data_complete_df[(school_data_complete_df['grade']=='10th')]
eleventh_graders = school_data_complete_df[(school_data_complete_df['grade']=='11th')]
twelfth_graders = school_data_complete_df[(school_data_complete_df['grade']=='12th')]

ninth_graders.head()

,Unnamed: 0,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
12,12,12,Brittney Walker,F,9th,Huang High School,64,79,0,District,2917,1910635
13,13,13,William Long,M,9th,Huang High School,71,79,0,District,2917,1910635


In [18]:
# Find average math and reading per school

ninth_grade_math_scores = ninth_graders.groupby('school_name').mean()['math_score']

tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]

eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]

twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]


ninth_grade_reading_scores = ninth_graders.groupby('school_name').mean()['reading_score']

tenth_grade_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]

eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]

twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

twelfth_grade_reading_scores

school_name
Bailey High School       80.912451
Cabrera High School      84.287958
Figueroa High School     81.384863
Ford High School         80.662338
Griffin High School      84.013699
Hernandez High School    80.857143
Holden High School       84.698795
Huang High School        80.305983
Johnson High School      81.227564
Pena High School         84.591160
Rodriguez High School    80.376426
Shelton High School      82.781671
Thomas High School       83.831361
Wilson High School       84.317673
Wright High School       84.073171
Name: reading_score, dtype: float64

In [19]:
math_scores_by_grade = pd.DataFrame({'9th':ninth_grade_math_scores
                                    ,'10th':tenth_grade_math_scores
                                    ,'11th':eleventh_grade_math_scores
                                    ,'12th':twelfth_grade_math_scores})

reading_scores_by_grade = pd.DataFrame({'9th':ninth_grade_reading_scores
                                       ,'10th':tenth_grade_reading_scores
                                       ,'11th':eleventh_grade_reading_scores
                                       ,'12th':twelfth_grade_reading_scores})

math_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [20]:
# Format each grade column

math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:.1f}".format)

math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:.1f}".format)

math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:.1f}".format)

math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:.1f}".format)

# Make sure the columns are in the correct order.
math_scores_by_grade = math_scores_by_grade[["9th", "10th", "11th", "12th"]]

# Remove the index name.
math_scores_by_grade.index.name = None
# Display the DataFrame.
math_scores_by_grade.head()

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4


In [21]:
# Format each grade column

reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:.1f}".format)

reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:.1f}".format)

reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:.1f}".format)

reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:.1f}".format)

# Make sure the columns are in the correct order.
raeding_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]

# Remove the index name.
reading_scores_by_grade.index.name = None
# Display the DataFrame.
reading_scores_by_grade.head()

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0


In [22]:
# Get description of budget to identify values for bins
per_school_capita

Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64

In [23]:
# Create bins for grouping schools by budget per student
spending_bins=[0,585,630,645,675]

# Name each bin
group_names = ['< $586',' $586-630',' $630-645',' $646-675']

# Identify which schools are in which bins
budget_groups = pd.cut(per_school_capita,spending_bins, labels=group_names)

# Identify how many schools are in each bin
per_school_capita.groupby(budget_groups).count()

< $586       4
 $586-630    4
 $630-645    4
 $646-675    3
dtype: int64

In [24]:
per_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [25]:
per_school_summary_df['Spending Ranges (Per Student)'] = pd.cut(per_school_capita,spending_bins,labels=group_names)

budget_summary = per_school_summary_df.groupby('Spending Ranges (Per Student)').mean()

budget_summary.drop('Total Students',inplace=True,axis=1)

In [26]:
# Calculate averages for the desired columns.
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]

spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]

spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]

spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]

overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]


In [27]:
# Assemble into DataFrame.
spending_summary_df = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores,
          "% Passing Math": spending_passing_math,
          "% Passing Reading": spending_passing_reading,
          "% Overall Passing": overall_passing_spending})

In [28]:
# Formatting
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)

spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)

spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}%".format)

spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}%".format)

spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}%".format)

spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
< $586,83.5,83.9,93%,97%,90%
$586-630,81.9,83.2,87%,93%,81%
$630-645,78.5,81.6,73%,84%,63%
$646-675,77.0,81.0,66%,81%,54%


In [29]:
# Create size bins and labels
size_bins = [0,999,1999,5000]
size_group_names = ['<1000','1000-1999','2000-5000']

# Apply bins + print
size_groups = pd.cut(per_school_counts,size_bins,labels=size_group_names)

In [30]:
# MY WAY -- find averages, then drop the 'Total Students' column
per_school_summary_df['School Size'] = size_groups

school_size_summary = per_school_summary_df.groupby('School Size').mean()

school_size_summary.drop('Total Students',inplace=True,axis=1)

In [31]:
# THEIR WAY -- Calculate averages for the desired columns.
size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]

size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]

size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]

size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]

size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

In [32]:
# Assemble into DataFrame.
size_summary_df = pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": size_overall_passing})

In [33]:
# Formatting.
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)

size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)

size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}%".format)

size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}%".format)

size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}%".format)

size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
<1000,83.8,83.9,94%,96%,90%
1000-1999,83.4,83.9,94%,97%,91%
2000-5000,77.7,81.3,70%,83%,58%


In [34]:
# Gather averages based on school type
type_math_scores=per_school_summary_df.groupby('School Type').mean()['Average Math Score']
type_reading_scores=per_school_summary_df.groupby('School Type').mean()['Average Reading Score']
type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

In [35]:
# Assemble into a dataframe
type_summary_df = pd.DataFrame({'Average Math Score':type_math_scores
                               ,'Average Reading Score':type_reading_scores
                               ,'% Passing Math':type_passing_math
                               ,'% Passing Reading':type_passing_reading
                               ,'% Overall Passing':type_overall_passing})

In [36]:
# Format the dataframe
type_summary_df['Average Math Score']=type_summary_df['Average Math Score'].map('{:.1f}'.format)
type_summary_df['Average Reading Score']=type_summary_df['Average Reading Score'].map('{:.1f}'.format)
type_summary_df['% Passing Math']=type_summary_df['% Passing Math'].map('{:.0f}%'.format)
type_summary_df['% Passing Reading']=type_summary_df['% Passing Reading'].map('{:.0f}%'.format)
type_summary_df['% Overall Passing']=type_summary_df['% Overall Passing'].map('{:.0f}%'.format)

type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,94%,97%,90%
District,77.0,81.0,67%,81%,54%


In [37]:
# Confirm metrics for Thomas High School
per_school_summary_df.loc['Thomas High School':'Thomas High School']

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.84893,93.272171,97.308869,90.948012,$630-645,1000-1999


In [43]:
thomas_ninth_only=student_data_df.loc[(student_data_df['school_name']=='Thomas High School') & (student_data_df['grade']=='9th')]
thomas_ninth_only['% Passing Math'].mean()

KeyError: '% Passing Math'